In [ ]:

import numpy as np
import tensorflow as tf
import pathlib
from pathlib import Path


file_list = None
train_ds = None
val_ds = None
test_ds = None
label_names = None
# audio = None
example_audio = None
example_labels = None
example_filenames = None



TRAIN_DIR = None
TEST_DIR = None
DATA_DIR = None
VAL_DIR = None


Divided into directories this way, you can easily load the data using `keras.utils.audio_dataset_from_directory`.

The audio clips are 1 second or less at 16kHz. The `output_sequence_length=16000` pads the short ones to exactly 1 second (and would trim longer ones) so that they can be easily batched.


In [ ]:
# def build():
#     global file_list
#     global train_ds
#     global val_ds
#     global label_names
#     file_list = tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False)
#     # file_list = sorted(tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False))

#     seconds=20
#     train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
#         directory=TRAIN_DIR,
#         batch_size=64,
#         validation_split=0.2,
#         seed=0,
#         output_sequence_length=16000*seconds,
#         subset='both'
#         ) 
  
#     label_names = np.array(train_ds.class_names)
#     # set_label_names(np.array(train_ds.class_names))
#     print()
#     print("label names:", label_names)
    
#     train_ds.element_spec

In [ ]:
def build():
    global file_list
    global train_ds
    global val_ds
    global label_names
    file_list = tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False)
    # file_list = sorted(tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False))

    seconds=20
    train_ds = tf.keras.utils.audio_dataset_from_directory(
        directory=TRAIN_DIR,
        batch_size=64,
        seed=0,
        output_sequence_length=16000*seconds,
        ) 
    val_ds = tf.keras.utils.audio_dataset_from_directory(
        directory=VAL_DIR,
        batch_size=64,
        seed=0,
        output_sequence_length=16000*seconds,
        ) 
  
    label_names = np.array(train_ds.class_names)
    # set_label_names(np.array(train_ds.class_names))
    print()
    print("train_ds label names:", label_names)
    print("val_ds label names:  ", np.array(val_ds.class_names))
    
    train_ds.element_spec

In [ ]:
# def build():
#     global file_list
#     global train_ds
#     global label_names
    
#     # Lade die Liste der Dateien ohne Shuffle
#     file_list = tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False)
    
#     # Definiere die Label-Namen basierend auf den Ordnernamen
#     label_mapping = {
#         'orig-16-44-mono': 0,
#         'upscale-from-mp3-128': 1
#     }
#     label_names = np.array(list(label_mapping.keys()))
    
#     # Funktion, um das Label aus dem Dateipfad zu extrahieren
#     def extract_label_from_path(file_path):
#         file_path_str = file_path.numpy().decode('utf-8')  # Konvertiere den Tensor zu einem String
#         parts = pathlib.Path(file_path_str).parts  # Teilt den Pfad in seine Teile auf
#         folder_name = parts[-2]  # Nimmt den Ordnernamen (zweitletztes Element im Pfad)
#         return label_mapping[folder_name]

#     # Verwende `map` mit einer Funktion, um den Tensor zu verarbeiten
#     def process_path(file_path):
#         label = tf.py_function(func=extract_label_from_path, inp=[file_path], Tout=tf.int64)
#         return file_path, label

#     # Mappe die Dateipfade zu ihren Labels
#     labeled_files = file_list.map(process_path)

#     # Lade die Audiodateien und verknüpfe sie mit den Labels
#     def load_wav(file_path, label):
#         audio = tf.io.read_file(file_path)
#         audio, _ = tf.audio.decode_wav(audio)
#         return audio, label
    
#     train_ds = labeled_files.map(lambda x, y: load_wav(x, y))
    
#     # Überprüfung der Zuordnung
#     print("Manuell zugewiesene Labels und Dateinamen:")
#     for file_path, label in labeled_files.take(10):
#         print(f"Dateiname: {file_path.numpy().decode('utf-8')}, Label: {label.numpy()}")

In [ ]:
# def build():
#     global file_list
#     global train_ds
#     global val_ds
#     global label_names
    
#     # Lade die Liste der Dateien ohne Shuffle
#     file_list = tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False)
    
#     # Definiere die Label-Namen basierend auf den Ordnernamen
#     label_mapping = {
#         'orig-16-44-mono': 0,
#         'upscale-from-mp3-128': 1
#     }
#     label_names = np.array(list(label_mapping.keys()))
    
#     # Funktion, um das Label aus dem Dateipfad zu extrahieren
#     def extract_label_from_path(file_path):
#         file_path_str = file_path.numpy().decode('utf-8')  # Konvertiere den Tensor zu einem String
#         parts = pathlib.Path(file_path_str).parts  # Teilt den Pfad in seine Teile auf
#         folder_name = parts[-2]  # Nimmt den Ordnernamen (zweitletztes Element im Pfad)
#         return label_mapping[folder_name]

#     # Verwende `map` mit einer Funktion, um den Tensor zu verarbeiten
#     def process_path(file_path):
#         label = tf.py_function(func=extract_label_from_path, inp=[file_path], Tout=tf.int64)
#         return file_path, label

#     # Mappe die Dateipfade zu ihren Labels
#     labeled_files = file_list.map(process_path)

#     # Lade die Audiodateien und verknüpfe sie mit den Labels
#     def load_wav(file_path, label):
#         audio = tf.io.read_file(file_path)
#         audio, _ = tf.audio.decode_wav(audio)
#         return audio, label

#     # Erstelle ein Dataset mit den Labels und Audiodaten
#     labeled_dataset = labeled_files.map(lambda x, y: load_wav(x, y))

#     # Aufteilen in Trainings- und Validierungsdatensätze
#     train_size = int(0.8 * len(label_names))  # 80% für Training
#     val_size = len(label_names) - train_size  # 20% für Validierung

#     # Erstelle das Trainings- und Validierungsdataset
#     train_ds = labeled_dataset.take(train_size)
#     val_ds = labeled_dataset.skip(train_size)

#     # Überprüfung der Zuordnung
#     print("Manuell zugewiesene Labels und Dateinamen:")
#     for file_path, label in labeled_files.take(10):
#         print(f"Dateiname: {file_path.numpy().decode('utf-8')}, Label: {label.numpy()}")
    
#     print("Trainingsdatensatz:", train_ds.element_spec)
#     print("Validierungsdatensatz:", val_ds.element_spec)

The dataset now contains batches of audio clips and integer labels. The audio clips have a shape of `(batch, samples, channels)`.


In [ ]:
# train_ds.element_spec

This dataset only contains single channel audio, so use the `tf.squeeze` function to drop the extra axis:


In [ ]:
def drop():
  global train_ds
  global val_ds
  @tf.autograph.experimental.do_not_convert
  def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

  train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
  val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

The `utils.audio_dataset_from_directory` function only returns up to two splits. It's a good idea to keep a test set separate from your validation set.
Ideally you'd keep it in a separate directory, but in this case you can use `Dataset.shard` to split the validation set into two halves. Note that iterating over **any** shard will load **all** the data, and only keep its fraction.


In [ ]:
def split():
    global test_ds
    global val_ds
    test_ds = val_ds.shard(num_shards=2, index=0)
    val_ds = val_ds.shard(num_shards=2, index=1)

In [ ]:
def extract():
  global example_audio
  global example_labels
  global example_filenames
  for example_audio, example_labels in train_ds.take(1):  
    print(example_audio.shape)
    print(example_labels.shape)
    
    # Extrahiere Dateinamen aus dem Dataset
  example_filenames = []
  for filepath in file_list.take(len(example_audio)):
      example_filenames.append(pathlib.Path(filepath.numpy().decode('utf-8')).name)

In [2]:
# def compare():
#   global example_filenames
#   global example_labels
#   global label_names
#   # Zähler für die tatsächlichen Labels
#   actual_labels_count = 0

#   # Konsolenausgabe zur Überprüfung der Zuordnung
#   for i in range(len(example_filenames)):
#       # Hole das Label aus example_labels
#       label_index = example_labels[i].numpy()  
#       expected_label = label_names[label_index]

#       # Extrahiere das Label aus dem Dateinamen
#       label_in_filename = example_filenames[i].split('_')[0]  # Hier anpassen, falls das Trennzeichen anders ist
      
#       # Zähle die tatsächlichen Labels
#       if label_in_filename == expected_label:
#           actual_labels_count += 1
#           if(label_in_filename =="upscale-16-48"):
#             print('-' * 50)
#             print(f"Index: {i}")
#             print(f"Dateiname: {example_filenames[i]}")
#             print(f"Label im Dateinamen: {label_in_filename}, Erwartetes Label: {expected_label}")
#       else:
#         print('!' * 50)
#         print(f"Index: {i}")
#         print(f"Dateiname: {example_filenames[i]}")
#         print(f"Label im Dateinamen: {label_in_filename}, Erwartetes Label: {expected_label}")

#   # Am Ende die Gesamtanzahl der Labels ausgeben
#   print('-' * 50)  
#   print('-' * 50)
#   print('-' * 50)
#   print(f"Anzahl der tatsächlichen Labels: {actual_labels_count}")
#   print(f"Anzahl der Labels im Dataset: {len(example_labels)}")

In [ ]:
def compare():
    global example_filenames
    global example_labels
    global label_names
    
    actual_labels_count = 0

    #Konsolenausgabe zur Überprüfung der Zuordnung
    for i in range(len(example_filenames)):
        # Hole das Label aus example_labels (prüfe, ob .numpy() notwendig ist)
        label_index = example_labels[i].numpy() if isinstance(example_labels[i], tf.Tensor) else example_labels[i]
        expected_label = label_names[label_index]

        # Extrahiere das Label aus dem Dateinamen (Anpassen, falls die Konvention anders ist)
        label_in_filename = example_filenames[i].split('_')[0]
        
        # Zähle die tatsächlichen Labels
        if label_in_filename == expected_label:
            actual_labels_count += 1
            if label_in_filename == "upscale-from-mp3-128":
                print('-' * 50)
                print(f"Index: {i}")
                print(f"Dateiname: {example_filenames[i]}")
                print(f"Label im Dateinamen: {label_in_filename}, Erwartetes Label: {expected_label}")
        else:
            print('!' * 50)
            print(f"Index: {i}")
            print(f"Dateiname: {example_filenames[i]}")
            print(f"Label im Dateinamen: {label_in_filename}, Erwartetes Label: {expected_label}")
            # Optional: Debugging-Info
            print(f"Vollstaendiger Dateipfad: {example_filenames[i]}")
    
    # Endergebnis ausgeben
    print('-' * 50)
    print(f"Anzahl der korrekten Labels: {actual_labels_count}")
    print(f"Anzahl der ueberprueften Labels: {len(example_labels)}")
    print('-' * 50)
    for i in range(len(example_filenames)):
        print(f"Dateiname: {example_filenames[i]}")
        print(f"Label: {label_names[example_labels[i]]}")

In [ ]:
# def run(input_file_list, input_train_ds, input_val_ds, input_test_ds, input_label_names, input_example_labels, input_example_audio, input_example_filenames):
def run(_TRAIN_DIR, _TEST_DIR, _DATA_DIR, _VAL_DIR):
    global TRAIN_DIR, TEST_DIR, DATA_DIR,VAL_DIR, file_list, example_labels, example_audio, train_ds, val_ds, test_ds, label_names, example_audio, example_labels, example_filenames

    TRAIN_DIR =_TRAIN_DIR
    TEST_DIR =_TEST_DIR
    DATA_DIR = _DATA_DIR
    VAL_DIR = _VAL_DIR
    # file_list = input_file_list
    # train_ds = input_train_ds
    # val_ds = input_val_ds
    # test_ds = input_test_ds
    # label_names = input_label_names
    # example_labels = input_example_labels
    # example_audio = input_example_audio
    # example_filenames = input_example_filenames

    build()
    drop()
    split()
    extract()
    compare()
    
    return file_list, example_labels, example_audio, train_ds, val_ds, test_ds, label_names, example_audio, example_labels, example_filenames